In [ ]:
import re
import os
import json
secrets = json.load(open('secrets.json'))
volume = '1'

In [8]:
lines= open(secrets['SCENE_DECK_PATH']+f'Scene Deck Volume {volume}.md').readlines()
# lines = [ l for l in lines.split('\n') if l]

In [9]:
grouped = []
curr = []
for l in lines:
    if l[0] == '-':
        grouped.append(curr)
        curr = [l]
    elif l[0] == ' ':
        curr.append(l)
grouped.append(curr)
grouped = grouped[1:]

In [10]:
icons_ref = {'Pokemon':'moon-orbit', 'Trainer':'person', 'World':'globe', 'Bonding':'opposite-hearts', 'Teams':'double-face-mask', 'Legendaries':'cross-flare'}
color_ref = {'Pokemon':'FireBrick', 'Trainer':'DarkBlue', 'World':'SaddleBrown', 'Bonding':'darkgreen', 'Teams':'black', 'Legendaries':'Goldenrod'}

built = []
for group in grouped:
    card = {}
    card['prompt'] = group[0].strip()[2:]
    # print(group)
    cate_tags = group[1].strip()[2:].split(' ', 1)
    card['category'] = cate_tags[0].strip().capitalize()
    card['tags'] = ','.join([f'"{t.strip()}"' for t in cate_tags[1].split(',')]) if len(cate_tags) > 1 else ''
    
    card['color'] = color_ref[card['category']]
    card['icon'] = icons_ref[card['category']]
    
    questions = re.findall('.*?\?',group[2].strip()[2:])
    card['questions'] = ['"bullet | '+q+'"' for q in questions]
    
    if len(group) > 3:
        encounter = group[3].strip()[2:].split(';')
        card['encounter'] = ['"bullet | '+e+'"' for e in encounter]
            
    built.append(card)

In [11]:
card_json = []
for b in built:
    encounters = '"section | Encounter",\n'+',\n'.join(b['encounter']) if 'encounter' in b else ''
    questions = ',\n'.join(b['questions'])
    if encounters: questions = questions+',\n'
    template = f""""count": 1,
        "title": "{b['category']}",
        "contents": [
          "section | Prompt",
          "text | {b['prompt']}",
          "section | Questions",
          {questions}
          {encounters}
        ],
        "tags": [{b['tags']}],
        "color": "{b['color']}",
         "icon": "{b['icon']}" """
    card_json.append("{\n"+template+"\n}")
out = '[\n'+',\n'.join(card_json)+'\n]'
os.makedirs(f"Volume {volume}",exist_ok=True)
with open(f'Volume {volume}/Scene Deck Volume {volume}.json', 'w') as f:
    f.write(out)
# https://crobi.github.io/rpg-cards/generator/generate.html <---- Now upload here